In [1]:
import numpy as np
import pandas as pd

from ast import literal_eval as lv
from data import PathPlanningCSVOperator

In [2]:
ALG = 'fmt'
MODE = 'evaluation'
HW = '0'

In [3]:
CSVOperator = PathPlanningCSVOperator(ALG, MODE, f'_hw_{HW}')
ddf = pd.read_csv(CSVOperator.fpath)

In [4]:
ddf

,Overall Test Number,Algorithm,Map Type,Map Id,Start Point,Goal Point,Test Number,Iteration,Timestep,Num Collision Checks,Batch Size,Cumulative Num Sampled,Current Path Cost,Any Path Found
0,10000,fmt,alternatinggaps,1,"[35, 29]","[39, 64]",0,82,3.78,142,1000,1000,39.04,True
1,10001,fmt,alternatinggaps,1,"[111, 219]","[240, 221]",1,457,16.13,508,1000,1000,132.77,True
2,10002,fmt,alternatinggaps,1,"[186, 150]","[243, 230]",2,371,15.49,507,1000,1000,107.14,True
3,10003,fmt,alternatinggaps,1,"[206, 66]","[177, 196]",3,401,15.11,507,1000,1000,140.01,True
4,10004,fmt,alternatinggaps,1,"[242, 76]","[222, 18]",4,108,4.63,167,1000,1000,65.48,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,10195,fmt,bugtrap,40,"[117, 249]","[239, 160]",0,570,19.60,642,1000,1000,154.48,True
196,10196,fmt,bugtrap,40,"[176, 215]","[40, 129]",1,645,23.18,737,1000,1000,164.81,True
197,10197,fmt,bugtrap,40,"[5, 132]","[215, 27]",2,818,27.63,908,1000,1000,255.41,True
198,10198,fmt,bugtrap,40,"[71, 6]","[243, 42]",3,98,6.40,116,1000,1000,-1.00,False


## Solution Cost vs Execution Time

In [6]:
np.linalg.norm(np.subtract(lv(ddf["Start Point"][0]), lv(ddf["Goal Point"][0])))

35.22782990761707

In [8]:
ed = list()
for i in range(len(ddf)):
    ed.append(np.linalg.norm(np.subtract(lv(ddf["Start Point"][i]), lv(ddf["Goal Point"][i]))))

In [10]:
ddf["norm_cost"] = ddf["Current Path Cost"]/ed

In [11]:
ddf

,Overall Test Number,Algorithm,Map Type,Map Id,Start Point,Goal Point,Test Number,Iteration,Timestep,Num Collision Checks,Batch Size,Cumulative Num Sampled,Current Path Cost,Any Path Found,norm_cost
0,10000,fmt,alternatinggaps,1,"[35, 29]","[39, 64]",0,82,3.78,142,1000,1000,39.04,True,1.108215
1,10001,fmt,alternatinggaps,1,"[111, 219]","[240, 221]",1,457,16.13,508,1000,1000,132.77,True,1.029101
2,10002,fmt,alternatinggaps,1,"[186, 150]","[243, 230]",2,371,15.49,507,1000,1000,107.14,True,1.090713
3,10003,fmt,alternatinggaps,1,"[206, 66]","[177, 196]",3,401,15.11,507,1000,1000,140.01,True,1.051163
4,10004,fmt,alternatinggaps,1,"[242, 76]","[222, 18]",4,108,4.63,167,1000,1000,65.48,True,1.067293
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,10195,fmt,bugtrap,40,"[117, 249]","[239, 160]",0,570,19.60,642,1000,1000,154.48,True,1.022957
196,10196,fmt,bugtrap,40,"[176, 215]","[40, 129]",1,645,23.18,737,1000,1000,164.81,True,1.024238
197,10197,fmt,bugtrap,40,"[5, 132]","[215, 27]",2,818,27.63,908,1000,1000,255.41,True,1.087836
198,10198,fmt,bugtrap,40,"[71, 6]","[243, 42]",3,98,6.40,116,1000,1000,-1.00,False,-0.005691


In [12]:
ddf["Current Path Cost"]/ddf["Iteration"]

0      0.476098
1      0.290525
2      0.288787
3      0.349152
4      0.606296
         ...   
195    0.271018
196    0.255519
197    0.312237
198   -0.010204
199    0.505271
Length: 200, dtype: float64

In [ ]:
(eval_df["Timestep"]/eval_df["Iteration"])*100

In [13]:
np.sqrt(2)*0.75

1.0606601717798214